In [34]:
import sys
import os
import pandas as pd
import nltk
import gensim.downloader as api
import wordninja
import wordhoard

TUT_CSV = 'Datasets/TUT18_train.csv'
SCAPPER_CSV = 'Datasets/scrapper_train_dataset.csv'
AUDIOSET_CSV = 'class_labels_indices.csv'

TUT_DF = pd.read_csv(TUT_CSV)
SCAPPER_DF = pd.read_csv(SCAPPER_CSV)
AUDIOSET_DF = pd.read_csv(AUDIOSET_CSV)


In [51]:
scapper_scenes = SCAPPER_DF['acoustic_scene_label'].unique()
tut_scenes = TUT_DF['labels'].unique()
audioset_events = AUDIOSET_DF['display_name'].to_list()

In [36]:
scapper_tut_csv = 'csv_files/scrapper_tut_scenes_cosineDistance.csv'
scapper_tut_df = pd.read_csv(scapper_tut_csv)

for i in list(scapper_tut_df.columns):
    for j in range(len(scapper_tut_df[i])):
        scapper_tut_df[i][j] = float(scapper_tut_df[i][j].strip('[]'))

In [48]:

from itertools import combinations
sample = 'openairmarket'
word_list = (wordninja.split(sample))

for item in combinations(word_list, 2):
    print(item)

#wordhoard.Synonyms(search_string='traffic').find_synonyms()


('open', 'air')
('open', 'market')
('air', 'market')


In [57]:
scapper_tut_df.index = scapper_scenes

for i in scapper_tut_df.columns:
    tut_scene_word = ' '.join(wordninja.split(i)).lower()
    #scapper_scene_word = ' '.join(wordninja.split(scapper_scenes[scapper_tut_df[i].argmin()])).lower()


    


In [58]:
scapper_tut_df

,airport,bus,metro,metro_station,park,public_square,shopping_mall,street_pedestrian,street_traffic,tram
bus,0.307345,0.0,0.389623,0.414777,0.34834,0.30922,0.506499,0.348946,0.365875,0.274673
busystreet,0.384625,0.31914,0.39756,0.422951,0.462198,0.340644,0.404535,0.292007,0.193514,0.369832
office,0.338082,0.338314,0.372395,0.349005,0.404996,0.346028,0.401228,0.465286,0.46136,0.313811
openairmarket,0.236437,0.393097,0.322114,0.318475,0.268742,0.238018,0.333853,0.310069,0.354952,0.355036
park,0.244575,0.34834,0.35438,0.357994,0.0,0.233535,0.455218,0.289462,0.39349,0.350273
quietstreet,0.463253,0.423177,0.448098,0.481156,0.392577,0.336434,0.541477,0.416671,0.423206,0.407499
restaurant,0.303431,0.455589,0.360011,0.364284,0.343734,0.361002,0.303786,0.36968,0.357405,0.40123
supermarket,0.273975,0.441031,0.323301,0.290754,0.376127,0.328261,0.251057,0.397173,0.389431,0.337374
tube,0.35932,0.27234,0.318456,0.346439,0.408849,0.307886,0.512232,0.443669,0.461726,0.178516
tubestation,0.355687,0.345289,0.369006,0.391809,0.431434,0.370215,0.510831,0.47936,0.467673,0.259143


In [28]:
import re
pattern = r'[^a-zA-Z0-9\s]'
event_label_list = []
scapper_unique_events = set()

for i in range (len(SCAPPER_DF['events_label_list'])):
    event_list = re.sub(pattern, '', SCAPPER_DF['events_label_list'][i])
    event_label_list.append(event_list.split(' '))

    scapper_unique_events.update(event_list.split(' '))

In [41]:
from sentence_transformers import SentenceTransformer
#language_model = SentenceTransformer('all-MiniLM-L6-v2')
#language_model = SentenceTransformer('nli-roberta-base')
language_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

#scapper_scenes_semantic_embeddings = language_model.encode(scapper_scenes)
#tut_scenes_semantic_embeddings = language_model.encode(tut_scenes)
#scapper_event_semantic_embeddings = language_model.encode(list(scapper_unique_events))
#audioset_event_semantic_embeddings = language_model.encode(audioset_events)


config_sentence_transformers.json: 100%|██████████| 212/212 [00:00<00:00, 391kB/s]
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [30]:
from sklearn.metrics.pairwise import cosine_distances

def get_embeddings_distance(label1, label2):
    semantic_embeddings1 = np.reshape(language_model.encode(label1), (1, -1))
    semantic_embeddings2 = np.reshape(language_model.encode(label2), (1, -1))
    return cosine_distances(semantic_embeddings1, semantic_embeddings2)

In [42]:
scapper_tut_scenes_df = pd.DataFrame(index=scapper_scenes, columns=tut_scenes)

for i in range(len(tut_scenes)):
    for j in range(len(scapper_scenes)):
        cosine_distanc = get_embeddings_distance(tut_scenes[i], scapper_scenes[j])
        scapper_tut_scenes_df[tut_scenes[i]][scapper_scenes[j]] = cosine_distanc


In [43]:
scapper_tut_scenes_df

,airport,bus,metro,metro_station,park,public_square,shopping_mall,street_pedestrian,street_traffic,tram
bus,[[0.44500858]],[[0.0]],[[0.44897103]],[[0.51701355]],[[0.570608]],[[0.6678418]],[[0.61372274]],[[0.5561604]],[[0.558812]],[[0.30302346]]
busystreet,[[0.6533866]],[[0.5191777]],[[0.6053889]],[[0.62173045]],[[0.6409673]],[[0.6748071]],[[0.4731632]],[[0.57769245]],[[0.49492645]],[[0.6314055]]
office,[[0.5602579]],[[0.6008407]],[[0.5779165]],[[0.6302589]],[[0.6095437]],[[0.71624917]],[[0.6533545]],[[0.72193676]],[[0.71870464]],[[0.618249]]
openairmarket,[[0.5228181]],[[0.6559249]],[[0.61030555]],[[0.6616736]],[[0.6791891]],[[0.6336493]],[[0.62425035]],[[0.7053625]],[[0.661092]],[[0.69111204]]
park,[[0.5190761]],[[0.570608]],[[0.59462684]],[[0.5799019]],[[0.0]],[[0.5889856]],[[0.52791846]],[[0.6186142]],[[0.6751635]],[[0.5460261]]
quietstreet,[[0.6883523]],[[0.65356624]],[[0.63787556]],[[0.66980696]],[[0.64475363]],[[0.6622069]],[[0.5501107]],[[0.6317034]],[[0.57718366]],[[0.6682646]]
restaurant,[[0.5630471]],[[0.5269964]],[[0.62955797]],[[0.62123626]],[[0.5365295]],[[0.69106764]],[[0.53193086]],[[0.6694387]],[[0.6790508]],[[0.6046682]]
supermarket,[[0.54287314]],[[0.54101336]],[[0.54825705]],[[0.5700641]],[[0.5473992]],[[0.6659119]],[[0.38583636]],[[0.6123327]],[[0.6605884]],[[0.5623369]]
tube,[[0.6722414]],[[0.6504031]],[[0.6807265]],[[0.6834584]],[[0.7263818]],[[0.7786652]],[[0.6775515]],[[0.75839853]],[[0.7261298]],[[0.62195754]]
tubestation,[[0.68347514]],[[0.6580833]],[[0.6589986]],[[0.627064]],[[0.67876834]],[[0.7562269]],[[0.62872773]],[[0.7588535]],[[0.734939]],[[0.64494157]]


In [44]:
scapper_tut_scenes_df.to_csv('scrapper_tut_scenes_02_cosineDistance.csv', index=False)

In [45]:
scapper_audioset_df = pd.DataFrame(index=audioset_events, columns=list(scapper_unique_events))

for i in range(len(list(scapper_unique_events))):
    for j in range(len(audioset_events)):
        cosine_distanc = get_embeddings_distance(list(scapper_unique_events)[i], audioset_events[j])
        scapper_audioset_df[list(scapper_unique_events)[i]][audioset_events[j]] = cosine_distanc


In [46]:
scapper_audioset_df.to_csv('scrapper_audioset_events_02_cosineDistance.csv', index=False)

In [1]:
import pandas as pd
import numpy as np
import os

In [47]:
scrapper_audioset_df = pd.read_csv('scrapper_audioset_events_02_cosineDistance.csv')
AUDIOSET_CSV = 'class_labels_indices.csv'
audioset_df = pd.read_csv(AUDIOSET_CSV)
audioset_eventLabels = audioset_df['display_name']
scrapper_audioset_mapping = {}
scrapper_events = (scrapper_audioset_df.columns).to_list()

for i in range(len(scrapper_events)):
    #print(audioset_eventLabels.iloc[scrapper_audioset_df[scrapper_events[i]].argmax()])
    scrapper_audioset_mapping[scrapper_events[i]] = audioset_eventLabels.iloc[scrapper_audioset_df[scrapper_events[i]].argmin()]

pd.DataFrame(scrapper_audioset_mapping, index=[0]).to_csv('scrapper_audioset_mapping_02.csv', index=False)

In [48]:
scrapper_audioset_mapping

{'lake': 'Water',
 'motorbike': 'Motorcycle',
 'mindthegap': 'Mantra',
 'speech': 'Female speech, woman speaking',
 'laughter': 'Giggle',
 'footstepsongrass': 'Walk, footsteps',
 'keylock': 'Computer keyboard',
 'switch': 'Gears',
 'trolley': 'Railroad car, train wagon',
 'keys': 'Computer keyboard',
 'knock': 'Engine knocking',
 'train': 'Rail transport',
 'cooking': 'Frying (food)',
 'cough': 'Throat clearing',
 'pushbike': 'Bicycle',
 'doorslam': 'Door',
 'buspassby': 'Bus',
 'checkoutbeeps': 'Yip',
 'drawer': 'Drawer open or close',
 'phone': 'Telephone',
 'doorclose': 'Door',
 'lightrain': 'Light engine (high frequency)',
 'running': 'Run',
 'gate': 'Door',
 'birdsong': 'Bird vocalization, bird call, bird song',
 'footsteps': 'Walk, footsteps',
 'money': 'Coin (dropping)',
 'clearthroat': 'Throat clearing',
 'slidingDoorclose': 'Sliding door',
 'wind': 'Wind noise (microphone)',
 'chairsmoving': 'Hands',
 'bagrustle': 'Bagpipes'}

In [49]:

import numpy as np

cosine_distances(np.reshape(scapper_scenes_semantic_embeddings[3], (1, -1)), np.reshape(tut_scenes_semantic_embeddings[2], (1, -1)))

array([[0.8425564]], dtype=float32)

In [50]:
scapper_scenes[3], tut_scenes[2]

('openairmarket', 'metro')